In [1]:
import numpy as np
import pandas as pd
import torch
import pickle
import matplotlib.pyplot as plt
from lpne.models import DcsfaNmf

MODEL_FILE = "/hpc/home/mk423/Anxiety/FullDataWork/Models/Final_mt_Model_500_epochs.pt"
DATA_PATH = "/work/mk423/CUS_EPM_Spr23/Sp23_CMS_Homecage_Data.pkl"
PROJECT_PATH = "/hpc/home/mk423/Anxiety/FullDataWork/Projections/"
FIGURE_PATH = "/hpc/home/mk423/Anxiety/FullDataWork/Figures/"
#cus_file = DATA_PATH + "CUS_data.pkl"
proj_file = PROJECT_PATH + "Sp23_HC_CUS_projection.csv"
mean_file = PROJECT_PATH + "Sp23_HC_CUS_mean_scores.csv"
model = torch.load(MODEL_FILE,map_location="cpu")
model.device="cpu"


#FEATURE_LIST = ['X_power_1_2','X_coh_1_2','X_gc_1_2']
#FEATURE_WEIGHT = [10,1,1]

import os, sys
umc_data_tools_path = "/hpc/home/mk423/Anxiety/Universal-Mouse-Code/"
sys.path.append(umc_data_tools_path)
import umc_data_tools as umc_dt

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/hpc/home/mk423/.local/lib/python3.7/site-packages/lpne/pipelines/__init__.py:14: UserWarning: Could not load lpne/pipelines/default_params.yaml!
  warnings.warn("Could not load lpne/pipelines/default_params.yaml!")


In [2]:
include_mice = [26352,
26354,
26361,
26362,
26363,
26364,
26371,
26374,
26382,
26384,
26385,
26391,
26392,
26393,
26394,]

In [3]:
cus_data_dict = pickle.load(open(DATA_PATH,'rb'))

X = cus_data_dict["X"]
y_mouse = cus_data_dict['y_mouse']
y_time = cus_data_dict['y_time']

In [4]:
i=0
remaining_mice = include_mice.copy()
for mouse in np.unique(y_mouse):
    if int(mouse[-5:]) in include_mice:
        mouse_mask = y_mouse==mouse
        remaining_mice.remove(int(mouse[-5:]))
        print(mouse,np.sum(mouse_mask))
        i+=1
        
print(i, remaining_mice)

Mouse26352 289
Mouse26354 260
Mouse26361 294
Mouse26362 295
Mouse26363 248
Mouse26364 254
Mouse26371 295
Mouse26374 255
Mouse26382 292
Mouse26384 292
Mouse26385 300
Mouse26391 300
Mouse26393 300
13 [26392, 26394]


In [5]:
s = model.project(X)
s_agg = s[:,:3] @ model.classifier[0].weight[0].detach().cpu().numpy()
agg_mean_scores = []
n1_mean_scores = []
n2_mean_scores = []
n3_mean_scores = []
mouse_list = []
for mouse in np.unique(y_mouse):
    if int(mouse[-5:]) in include_mice:
        mouse_mask = y_mouse==mouse
        mouse_list.append(mouse)
        agg_mean_scores.append(np.mean(s_agg[mouse_mask==1]))
        n1_mean_scores.append(np.mean(s[mouse_mask==1,0]))
        n2_mean_scores.append(np.mean(s[mouse_mask==1,1]))
        n3_mean_scores.append(np.mean(s[mouse_mask==1,2]))

In [7]:
df = pd.DataFrame.from_dict({
    "mouse":mouse_list,
    "agg_score":agg_mean_scores,
    "n1_score":n1_mean_scores,
    "n2_score":n2_mean_scores,
    "n3_score":n3_mean_scores,
})

df.to_csv(PROJECT_PATH + "Sp23_CMS_HC_mean_scores_corrected_chan_IDs.csv")